## 01_解析雨量_連続画像出力

### 概要：
ArcGIS Pro のプロジェクトのアクティブなマップで、気象データ変換ツールで変換した解析雨量のラスターレイヤーを、
表示状態を1レイヤーごとに切り替えて、画像を連続出力するサンプルです。

 - 事前に画像出力される際の、ラスターレイヤーのシンボル、レイヤーのブレンド、透過表示などは調整しておく必要があります。

## 連続画像出力のメイン部分

### 変数定義（↓適宜修正してご利用下さい）

In [13]:
# 変数定義
img_folder = r"ExportImg" # 画像を出力するフォルダー名（現在のプロジェクト下にフォルダーが存在しない場合、自動的に作成）
px_width = 1000 # 出力する png画像の幅（ピクセル数）
px_height = 1000 # 出力する png画像の高さ（ピクセル数）

### 前処理

In [14]:
import os
import sys
import arcpy

# 関数定義
def anal_filename_to_short_filename(fullname):
    """
    デフォルトで出力されるファイル名が長いので、次のように短縮したファイル名に変更
    例）
    Z__C_RJTD_20160103233000_SRF_GPV_Ggis1km_Prr60lv_ANAL_grib2.tif
    →
    Z__C_RJTD_20160103_2330_ANAL_grib2
    """
    shortname = fullname[0:18] + "_" + fullname[18:22] + "_" + fullname[-14:-4]
    return shortname

# 現在のプロジェクトからmap と activeview を取得
aprx = arcpy.mp.ArcGISProject("current")
m = aprx.activeMap
mv = aprx.activeView
# 画像を出力するフォルダーを、現在のプロジェクト直下に指定
export_folder = os.path.join(aprx.homeFolder, img_folder)
# フォルダーが存在しない場合は新規に作成
if os.path.exists(export_folder) == False:
    os.mkdir(export_folder)

### 解析雨量のレイヤー名 "Z__C_RJTD_*" でフィルターし一覧を取得

In [15]:
# 変換時のデフォルト設定の解析雨量のファイル名を想定
# 変換時やマップに読み込み後にレイヤー名変更している場合は引数を要変更
anal_layers = m.listLayers("Z__C_RJTD_*")

### 全てのレイヤーを表示をオフに設定後、レイヤー1つを表示状態にしながら画像出力します

In [16]:
# 一旦、全てのレイヤーの表示状態をオフに設定
for anal_layer in anal_layers:
    anal_layer.visible = False

In [17]:
# レイヤーを一つだけ表示状態に切替ながら画像を連続で出力
cnt = len(anal_layers)
for i in range(0, cnt):
    anal_layer = anal_layers[i]
    if i == 0:
        anal_layers[i].visible = True
    else:
        anal_layers[i-1].visible = False
        anal_layers[i].visible = True
    export_fname = anal_filename_to_short_filename(anal_layer.name) + ".png"
    export_filename = os.path.join(export_folder, export_fname)
    if arcpy.Exists(export_filename):
        arcpy.AddWarning(u"{} はすでに存在しているため出力をスキップします".format(export_fname))
    else:
        mv.exportToPNG(export_filename, width=px_width, height=px_height)
    if i == cnt-1:
        anal_layers[i].visible = False